# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse  Shipments          Customer  \
0  Sep  1 2022 11:04AM  245521        10  MSP213623  Methapharm, Inc.   
1  Sep  1 2022 11:04AM  245521        10  MSP213624  Methapharm, Inc.   
2  Sep  1 2022 11:04AM  245521        10  MSP213625  Methapharm, Inc.   
3  Sep  1 2022 11:04AM  245521        10  MSP213626  Methapharm, Inc.   
4  Sep  1 2022 11:04AM  245521        10  MSP213627  Methapharm, Inc.   
5  Sep  1 2022 11:04AM  245521        10  MSP213628  Methapharm, Inc.   
6  Sep  1 2022 11:04AM  245521        10  MSP213629  Methapharm, Inc.   
7  Sep  1 2022 11:04AM  245521        10  MSP213622  Methapharm, Inc.   
8  Sep  1 2022 11:04AM  245521        10  MSP213630  Methapharm, Inc.   
9  Sep  1 2022 11:04AM  245521        10  MSP213631  Methapharm, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
18  245515       Released          1
19  245517       Released          9
20  245519       Released         49
21  245520       Released          1
22  245521       Released         10

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
245515                NaN       1.0
245517                NaN       9.0
245519                NaN      49.0
245520                NaN       1.0
245521                NaN      10.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
245453                0.0      10.0
245470                0.0       1.0
245478               11.0      30.0
245479                0.0       1.0
245480                0.0       4.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
245453                  0        10
245470                  0         1
245478                 11        30
245479                  0         1
245480                  0         4

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               245453          0        10
1               245470          0         1
2               245478         11        30
3               245479          0         1
4               245480          0         4

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               245453                 10
1               245470                  1
2               245478        11       30
3               245479                  1
4               245480                  4

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                  Customer
0    Sep  1 2022 11:04AM  245521        10          Methapharm, Inc.
10   Sep  1 2022 10:59AM  245519        10         ISDIN Corporation
59   Sep  1 2022 10:55AM  245453        20  ACI Healthcare USA, Inc.
69   Sep  1 2022 10:55AM  245520        21         NBTY Global, Inc.
70   Sep  1 2022 10:41AM  245517        10         ISDIN Corporation
79   Sep  1 2022 10:39AM  245515        10         ISDIN Corporation
80   Sep  1 2022 10:31AM  245514        10                 SugarBear
81   Sep  1 2022 10:19AM  245513        10          Eywa Pharma Inc.
116  Sep  1 2022 10:19AM  245478        15     Alliance Pharma, Inc.
157  Sep  1 2022 10:13AM  245512        10          Methapharm, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                  Customer Executing  \
0  Sep  1 2022 11:04AM  245521        10          Methapharm, Inc.             
1  Sep  1 2022 10:59AM  245519        10         ISDIN Corporation             
2  Sep  1 2022 10:55AM  245453        20  ACI Healthcare USA, Inc.             
3  Sep  1 2022 10:55AM  245520        21         NBTY Global, Inc.             
4  Sep  1 2022 10:41AM  245517        10         ISDIN Corporation             
5  Sep  1 2022 10:39AM  245515        10         ISDIN Corporation             
6  Sep  1 2022 10:31AM  245514        10                 SugarBear             
7  Sep  1 2022 10:19AM  245513        10          Eywa Pharma Inc.             
8  Sep  1 2022 10:19AM  245478        15     Alliance Pharma, Inc.        11   
9  Sep  1 2022 10:13AM  245512        10          Methapharm, Inc.             

  Released  
0       10  
1       49  
2       10  
3        1  
4        9  
5        1  
6        1  
7       35  
8       30  
9        1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                  Customer Released  \
0  Sep  1 2022 11:04AM  245521        10          Methapharm, Inc.       10   
1  Sep  1 2022 10:59AM  245519        10         ISDIN Corporation       49   
2  Sep  1 2022 10:55AM  245453        20  ACI Healthcare USA, Inc.       10   
3  Sep  1 2022 10:55AM  245520        21         NBTY Global, Inc.        1   
4  Sep  1 2022 10:41AM  245517        10         ISDIN Corporation        9   
5  Sep  1 2022 10:39AM  245515        10         ISDIN Corporation        1   
6  Sep  1 2022 10:31AM  245514        10                 SugarBear        1   
7  Sep  1 2022 10:19AM  245513        10          Eywa Pharma Inc.       35   
8  Sep  1 2022 10:19AM  245478        15     Alliance Pharma, Inc.       30   
9  Sep  1 2022 10:13AM  245512        10          Methapharm, Inc.        1   

  Executing  
0            
1            
2            
3            
4            
5            
6            
7            
8        11  
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                  Customer Released  \
0  Sep  1 2022 11:04AM  245521        10          Methapharm, Inc.       10   
1  Sep  1 2022 10:59AM  245519        10         ISDIN Corporation       49   
2  Sep  1 2022 10:55AM  245453        20  ACI Healthcare USA, Inc.       10   
3  Sep  1 2022 10:55AM  245520        21         NBTY Global, Inc.        1   
4  Sep  1 2022 10:41AM  245517        10         ISDIN Corporation        9   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                  Customer  Released  \
0  Sep  1 2022 11:04AM  245521        10          Methapharm, Inc.      10.0   
1  Sep  1 2022 10:59AM  245519        10         ISDIN Corporation      49.0   
2  Sep  1 2022 10:55AM  245453        20  ACI Healthcare USA, Inc.      10.0   
3  Sep  1 2022 10:55AM  245520        21         NBTY Global, Inc.       1.0   
4  Sep  1 2022 10:41AM  245517        10         ISDIN Corporation       9.0   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Sep  1 2022 11:04AM  245521        10          Methapharm, Inc.      10.0   
1  Sep  1 2022 10:59AM  245519        10         ISDIN Corporation      49.0   
2  Sep  1 2022 10:55AM  245453        20  ACI Healthcare USA, Inc.      10.0   
3  Sep  1 2022 10:55AM  245520        21         NBTY Global, Inc.       1.0   
4  Sep  1 2022 10:41AM  245517        10         ISDIN Corporation       9.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3928107     211.0       14.0
15          981920      60.0       11.0
20          245453      10.0        0.0
21          245520       1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3928107     211.0       14.0
1        15   981920      60.0       11.0
2        20   245453      10.0        0.0
3        21   245520       1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     211.0       14.0
1        15      60.0       11.0
2        20      10.0        0.0
3        21       1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released  211.0
1        15   Released   60.0
2        20   Released   10.0
3        21   Released    1.0
4        10  Executing   14.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    15    20   21
Status                           
Executing   14.0  11.0   0.0  0.0
Released   211.0  60.0  10.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    15    20   21
0          Executing   14.0  11.0   0.0  0.0
1           Released  211.0  60.0  10.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    15    20   21
0  Executing   14.0  11.0   0.0  0.0
1   Released  211.0  60.0  10.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()